In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 8245451670452344457, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 9214062756
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 2394401572258738481
 physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"]

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, partial, rand, space_eval
from sklearn.metrics import log_loss
import sys
import time
import math
import os
import pandas as pd
from keras.callbacks import EarlyStopping
import numpy as np
from glob import glob
import cv2
import skimage
from skimage.transform import resize
from keras.utils.np_utils import to_categorical
import keras
from keras import layers
from keras import models
from keras import optimizers
from keras.models import load_model
# import keras.callbacks as kcall
from keras.optimizers import Adam, RMSprop,SGD
from keras.models import Model
from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
# from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.applications.vgg19 import VGG19
from keras.regularizers import l2

import matplotlib.pyplot as plt
from keras.layers import Input, concatenate
from keras import optimizers, metrics, models
from keras.layers import Input, Flatten, Dense

%matplotlib inline

Using TensorFlow backend.


In [4]:
import keras
import tensorflow as tf

print("Keras Version", keras.__version__)
print("tensorflow Version", tf.__version__)
# print("dim_ordering:", K.image_dim_ordering())

Keras Version 2.2.4
tensorflow Version 1.9.0


In [5]:
batch_size = 32
img_height, img_width = 128, 128
input_shape = (img_height, img_width, 3)
epochs = 15

In [6]:
train_dir = 'data/train/'
test_dir = 'data/test'

In [7]:
random_seed = np.random.seed(1142)

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    featurewise_center=True,
    featurewise_std_normalization=True,
    validation_split= 0.3,
    zoom_range=0.2,
    shear_range=0.2)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    subset = 'training',
    class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    subset = 'validation',
    class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    class_mode='categorical')

Found 34909 images belonging to 6 classes.
Found 14958 images belonging to 6 classes.
Found 13578 images belonging to 6 classes.


In [8]:
nb_train_samples = len(train_generator.filenames)
nb_validation_samples = len(validation_generator.filenames)
nb_test_samples = len(test_generator.filenames)

predict_size_train = int(math.ceil(nb_train_samples / batch_size))
predict_size_validation = int(math.ceil(nb_validation_samples / batch_size))
predict_size_test = int(math.ceil(nb_test_samples / batch_size))

num_classes = len(train_generator.class_indices)

print("nb_train_samples:", nb_train_samples)
print("nb_validation_samples:", nb_validation_samples)
print("nb_test_samples:", nb_test_samples)

print("\npredict_size_train:", predict_size_train)
print("predict_size_validation:", predict_size_validation)
print("predict_size_test:", predict_size_test)

print("\n num_classes:", num_classes)

nb_train_samples: 34909
nb_validation_samples: 14958
nb_test_samples: 13578

predict_size_train: 1091
predict_size_validation: 468
predict_size_test: 425

 num_classes: 6


In [10]:
# vgg19_weights ="../input/full-keras-pretrained-no-top/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5"
# inception_weights ="../input/full-keras-pretrained-no-top//inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
# vgg16_weights ="../input/full-keras-pretrained-no-top/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"
# denseNet201_weights ="../input/full-keras-pretrained-no-top/densenet201_weights_tf_dim_ordering_tf_kernels_notop.h5"
# denseNet121_weights ="../input/full-keras-pretrained-no-top/densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5"
# resenet50_weights ="../input/full-keras-pretrained-no-top/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"
# inception_resnet_v2_weights ="../input/full-keras-pretrained-no-top/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5"
# nasnet_weights ="../input/full-keras-pretrained-no-top/nasnet_large_no_top.h5"
# nasnet_mobile_weights ="../input/full-keras-pretrained-no-top/nasnet_mobile_no_top.h5"
# mobilenet_weights ="../input/full-keras-pretrained-no-top/mobilenet_1_0_224_tf_no_top.h5"

In [11]:
# from keras.applications.vgg19 import VGG19
# from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
# from keras.applications.xception import Xception, preprocess_input
# from keras.applications import DenseNet201
# from keras.applications import DenseNet121
# from keras.applications import ResNet50
# from keras.applications.inception_resnet_v2 import InceptionResNetV2
# from keras.applications import NASNetLarge, NASNetMobile
# from keras.applications import MobileNet

bottleneck_final_model=VGG16(weights="imagenet", include_top=False, pooling = "avg")

In [12]:
os.mkdir("extracted_features")
extracted_features_dir = "extracted_features/"
model_name = "vgg16_descriptors"

In [13]:
bottleneck_features_train = bottleneck_final_model.predict_generator(train_generator, predict_size_train, max_q_size=1, pickle_safe=False)
np.save(extracted_features_dir+'bottleneck_features_train_'+model_name+'.npy', bottleneck_features_train)

In [14]:
bottleneck_features_validation = bottleneck_final_model.predict_generator(validation_generator, predict_size_validation)
np.save(extracted_features_dir+'bottleneck_features_validation_'+model_name+'.npy', bottleneck_features_validation)

In [15]:
bottleneck_features_test = bottleneck_final_model.predict_generator(test_generator, predict_size_test)
np.save(extracted_features_dir+'bottleneck_features_test_'+model_name+'.npy', bottleneck_features_test)

In [16]:

import tensorflow as tf
from keras.backend.tensorflow_backend import get_session
from keras.backend.tensorflow_backend import clear_session
from keras.backend.tensorflow_backend import set_session

def reset_keras_tf_session():
    """
    this function clears the gpu memory and set the 
    tf session to not use the whole gpu
    """
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

#     config = tf.ConfigProto()
#     config.gpu_options.allow_growth = True
#     set_session(tf.Session(config=config))

reset_keras_tf_session()

In [17]:
train_data = np.load(extracted_features_dir+'bottleneck_features_train_'+model_name+'.npy')
validation_data = np.load(extracted_features_dir+'bottleneck_features_validation_'+model_name+'.npy')
test_data = np.load(extracted_features_dir+'bottleneck_features_test_'+model_name+'.npy')

train_labels = train_generator.classes
train_labels = to_categorical(train_labels, num_classes=num_classes)

validation_labels = validation_generator.classes
validation_labels = to_categorical(validation_labels, num_classes=num_classes)

test_labels = test_generator.classes
test_labels = to_categorical(test_labels, num_classes=num_classes)

In [18]:
space = {
         'lr': hp.choice('lr',[0.1, 0.01, 0.001, 0.0001, 0.00001,0.000001, 0.0000001]),
#          'dropout': hp.choice('dropout', [0.4, 0.5, 0.6, 0.7]),
#          'batch_size': hp.choice('batch_size', [64]),
#          'epochs': hp.choice('epochs', [15, 20, 25, 30, 50]),
#          'optimizer': hp.choice('optimizer',['sgd','adam','rmsprop']),
#          'optimizer': hp.choice('optimizer',['rmsprop']),
#          'optimizer': hp.choice('optimizer',['adam']),
         'beta_1':hp.choice('beta_1',[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8, 0.9]),
         'beta_2':hp.choice('beta_2',[0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.999,0.99,0.995]),
#          'momentum':hp.choice('momentum',[0.3,0.5,0.7,0.9,1]),
#          'amsgrad':hp.choice('amsgrad',[False,True]),
#          'nesterov':hp.choice('nesterov',[False,True]),
#          'rho':hp.choice('rho',[0.4,0.5,0.6,0.7,0.8,0.9,1]),
        }

In [19]:
from sklearn.metrics import accuracy_score
def f_nn(params):   
    print ('Parameters testing: ', params)
  
    adam_opt=Adam(lr=params["lr"], beta_1=params["beta_1"], beta_2=params['beta_2'], epsilon=None, decay=0.0)
#     sgd=SGD(lr=params["lr"], momentum=params['momentum'], decay=0.0, nesterov=params['nesterov'])
#     rmsprop=RMSprop(lr=params["lr"], rho=params['rho'], epsilon=None, decay=0.0)

    model = Sequential()
    # model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.7))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer=adam_opt, loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(train_data, train_labels,
                        epochs=epochs,
                        batch_size=batch_size,
                        validation_data=(validation_data, validation_labels),
                        verbose= 2,
                        callbacks=get_callbacks(params))

    (eval_loss, eval_accuracy) = model.evaluate(validation_data, validation_labels, batch_size= batch_size, verbose=1)

    print("Validation Accuracy: {:.4f}%".format(eval_accuracy * 100))
    print("Validation Loss: {}".format(eval_loss))
    
    filename = test_generator.filenames
    truth = test_generator.classes
    label = test_generator.class_indices
    indexlabel = dict((value, key) for key, value in label.items())

    preds = model.predict(test_data)

    predictions = [i.argmax() for i in preds]
    y_true = [i.argmax() for i in test_labels]
#     cm = confusion_matrix(y_pred=predictions, y_true=y_true)

    print('Test Accuracy: {}'.format(accuracy_score(y_true=y_true, y_pred=predictions)))

    print("*_*" * 50)
#     best_epoch = np.argmax(history.history['val_acc'])
#     best_val_acc = np.max(history.history['val_acc'])
#     print('Epoch {} - val acc: {}'.format(best_epoch, best_val_acc))
    sys.stdout.flush() 
    
    return {'loss': eval_loss, 'status': STATUS_OK, 'model': model}

In [20]:
def get_callbacks(params):
    callbacks =[EarlyStopping(monitor='val_acc', patience=3, verbose=1)]
    return callbacks

In [21]:
trials = Trials()
best = fmin(f_nn, space, algo=tpe.suggest, max_evals = 300, trials=trials)
print(best)

Parameters testing:  {'beta_1': 0.2, 'lr': 0.001, 'beta_2': 0.7}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 5s - loss: 1.2236 - acc: 0.5069 - val_loss: 1.4711 - val_acc: 0.3378
Epoch 2/15
 - 4s - loss: 1.0663 - acc: 0.5766 - val_loss: 1.3940 - val_acc: 0.3696
Epoch 3/15
 - 4s - loss: 1.0195 - acc: 0.5999 - val_loss: 1.3555 - val_acc: 0.3820
Epoch 4/15
 - 4s - loss: 0.9918 - acc: 0.6127 - val_loss: 1.4162 - val_acc: 0.3619
Epoch 5/15
 - 4s - loss: 0.9717 - acc: 0.6259 - val_loss: 1.3411 - val_acc: 0.4660
Epoch 6/15
 - 4s - loss: 0.9599 - acc: 0.6331 - val_loss: 1.2683 - val_acc: 0.4323
Epoch 7/15
 - 4s - loss: 0.9563 - acc: 0.6354 - val_loss: 1.2828 - val_acc: 0.4246
Epoch 8/15
 - 4s - loss: 0.9512 - acc: 0.6362 - val_loss: 1.3820 - val_acc: 0.4128
Epoch 00008: early stopping
14958/14958 [==============================] - 0s 33us/step
Validation Accuracy: 41.2756%
Validation Loss: 1.3819525969167388
Test Accuracy: 0.5932390631904552
*_**_**_**_**_**_**_**_**_**_**_*

Parameters testing:  {'beta_1': 0.1, 'lr': 1e-06, 'beta_2': 0.7}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 4s - loss: 2.1988 - acc: 0.1705 - val_loss: 1.8506 - val_acc: 0.2264
Epoch 2/15
 - 4s - loss: 2.0955 - acc: 0.1865 - val_loss: 1.7781 - val_acc: 0.2264
Epoch 3/15
 - 4s - loss: 2.0172 - acc: 0.1993 - val_loss: 1.7279 - val_acc: 0.2276
Epoch 4/15
 - 4s - loss: 1.9584 - acc: 0.2111 - val_loss: 1.6944 - val_acc: 0.2458
Epoch 5/15
 - 4s - loss: 1.9155 - acc: 0.2180 - val_loss: 1.6734 - val_acc: 0.2726
Epoch 6/15
 - 4s - loss: 1.8858 - acc: 0.2233 - val_loss: 1.6603 - val_acc: 0.2853
Epoch 7/15
 - 4s - loss: 1.8626 - acc: 0.2305 - val_loss: 1.6521 - val_acc: 0.2905
Epoch 8/15
 - 4s - loss: 1.8375 - acc: 0.2345 - val_loss: 1.6463 - val_acc: 0.2932
Epoch 9/15
 - 4s - loss: 1.8173 - acc: 0.2393 - val_loss: 1.6420 - val_acc: 0.2869
Epoch 10/15
 - 4s - loss: 1.8037 - acc: 0.2451 - val_loss: 1.6385 - val_acc: 0.3308
Epoch 11/15
 - 4s - loss: 1.7877 - acc: 0.2511 - val_l

Parameters testing:  {'beta_1': 0.3, 'lr': 1e-06, 'beta_2': 0.6}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 5s - loss: 2.1359 - acc: 0.1853 - val_loss: 1.8165 - val_acc: 0.2264
Epoch 2/15
 - 4s - loss: 2.0434 - acc: 0.1976 - val_loss: 1.7515 - val_acc: 0.2264
Epoch 3/15
 - 4s - loss: 1.9678 - acc: 0.2073 - val_loss: 1.7126 - val_acc: 0.2264
Epoch 4/15
 - 4s - loss: 1.9224 - acc: 0.2190 - val_loss: 1.6886 - val_acc: 0.2264
Epoch 00004: early stopping
14958/14958 [==============================] - 1s 35us/step
Validation Accuracy: 22.6367%
Validation Loss: 1.6885503518069742
Test Accuracy: 0.229194284872588
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_1': 0.1, 'lr': 1e-05, 'beta_2': 0.5}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 5s - loss: 1.8753 - acc: 0.2215 - val_loss: 1.6327 - val_acc: 0.2938
Epoch 2/15
 - 4s 

 - 4s - loss: 1.1445 - acc: 0.5607 - val_loss: 1.4909 - val_acc: 0.2880
Epoch 5/15
 - 4s - loss: 1.1124 - acc: 0.5750 - val_loss: 1.4850 - val_acc: 0.2779
Epoch 6/15
 - 4s - loss: 1.0822 - acc: 0.5875 - val_loss: 1.4472 - val_acc: 0.3123
Epoch 7/15
 - 4s - loss: 1.0610 - acc: 0.5959 - val_loss: 1.4210 - val_acc: 0.3402
Epoch 8/15
 - 4s - loss: 1.0412 - acc: 0.6032 - val_loss: 1.4300 - val_acc: 0.3385
Epoch 9/15
 - 4s - loss: 1.0243 - acc: 0.6094 - val_loss: 1.4404 - val_acc: 0.3404
Epoch 10/15
 - 4s - loss: 1.0123 - acc: 0.6143 - val_loss: 1.4330 - val_acc: 0.3578
Epoch 11/15
 - 4s - loss: 0.9975 - acc: 0.6197 - val_loss: 1.3499 - val_acc: 0.3781
Epoch 12/15
 - 4s - loss: 0.9874 - acc: 0.6265 - val_loss: 1.3683 - val_acc: 0.3743
Epoch 13/15
 - 4s - loss: 0.9750 - acc: 0.6319 - val_loss: 1.3563 - val_acc: 0.3831
Epoch 14/15
 - 4s - loss: 0.9649 - acc: 0.6368 - val_loss: 1.3489 - val_acc: 0.3918
Epoch 15/15
 - 4s - loss: 0.9569 - acc: 0.6396 - val_loss: 1.3685 - val_acc: 0.3751
14958/149

Epoch 2/15
 - 4s - loss: 1.7277 - acc: 0.2730 - val_loss: 1.6227 - val_acc: 0.2875
Epoch 3/15
 - 4s - loss: 1.6105 - acc: 0.3279 - val_loss: 1.6076 - val_acc: 0.2775
Epoch 4/15
 - 4s - loss: 1.5327 - acc: 0.3833 - val_loss: 1.5974 - val_acc: 0.2498
Epoch 00004: early stopping
14958/14958 [==============================] - 1s 37us/step
Validation Accuracy: 24.9833%
Validation Loss: 1.597430223209811
Test Accuracy: 0.4034467520989837
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_1': 0.7, 'lr': 0.001, 'beta_2': 0.99}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 5s - loss: 1.2290 - acc: 0.5107 - val_loss: 1.4541 - val_acc: 0.3182
Epoch 2/15
 - 4s - loss: 1.0557 - acc: 0.5815 - val_loss: 1.4466 - val_acc: 0.3432
Epoch 3/15
 - 4s - loss: 1.0027 - acc: 0.6041 - val_loss: 1.3631 - val_acc: 0.3716
Epoch 4/15
 - 4s - loss: 0.9735 - acc: 0.6164 - 

...

Epoch 00012: early stopping
14958/14958 [==============================] - 2s 142us/step
Validation Accuracy: 36.9769%
Validation Loss: 1.396939082633789
Test Accuracy: 0.5832965090587715
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_1': 0.9, 'lr': 0.001, 'beta_2': 0.9}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 19s - loss: 1.2276 - acc: 0.5095 - val_loss: 1.4330 - val_acc: 0.3609
Epoch 2/15
 - 11s - loss: 1.0576 - acc: 0.5803 - val_loss: 1.3525 - val_acc: 0.3876
Epoch 3/15
 - 9s - loss: 1.0039 - acc: 0.6061 - val_loss: 1.3334 - val_acc: 0.3983
Epoch 4/15
 - 9s - loss: 0.9792 - acc: 0.6195 - val_loss: 1.4081 - val_acc: 0.3700
Epoch 5/15
 - 9s - loss: 0.9577 - acc: 0.6273 - val_loss: 1.3342 - val_acc: 0.3908
Epoch 6/15
 - 9s - loss: 0.9389 - acc: 0.6332 - val_loss: 1.3171 - val_acc: 0.3712
Epoch 00006: early stopping
14958/14958 [=====

Epoch 2/15
 - 8s - loss: 1.0671 - acc: 0.5813 - val_loss: 1.3692 - val_acc: 0.3686
Epoch 3/15
 - 9s - loss: 1.0316 - acc: 0.5965 - val_loss: 1.4074 - val_acc: 0.3771
Epoch 4/15
 - 8s - loss: 1.0116 - acc: 0.6067 - val_loss: 1.3928 - val_acc: 0.3779
Epoch 5/15
 - 9s - loss: 1.0050 - acc: 0.6097 - val_loss: 1.3542 - val_acc: 0.3767
Epoch 6/15
 - 10s - loss: 0.9859 - acc: 0.6171 - val_loss: 1.3651 - val_acc: 0.3720
Epoch 7/15
 - 9s - loss: 0.9773 - acc: 0.6239 - val_loss: 1.3025 - val_acc: 0.4226
Epoch 8/15
 - 9s - loss: 0.9910 - acc: 0.6159 - val_loss: 1.3650 - val_acc: 0.3903
Epoch 9/15
 - 10s - loss: 0.9836 - acc: 0.6177 - val_loss: 1.3252 - val_acc: 0.3963
Epoch 10/15
 - 9s - loss: 0.9736 - acc: 0.6252 - val_loss: 1.3428 - val_acc: 0.3915
Epoch 00010: early stopping
14958/14958 [==============================] - 2s 150us/step
Validation Accuracy: 39.1496%
Validation Loss: 1.3427735187249266
Test Accuracy: 0.5821917808219178
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**

 - 10s - loss: 1.5358 - acc: 0.3810 - val_loss: 1.5965 - val_acc: 0.2741
Epoch 00004: early stopping
14958/14958 [==============================] - 2s 129us/step
Validation Accuracy: 27.4101%
Validation Loss: 1.5965144863930723
Test Accuracy: 0.42207983502724994
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_1': 0.9, 'lr': 0.001, 'beta_2': 0.9}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 20s - loss: 1.2156 - acc: 0.5116 - val_loss: 1.4358 - val_acc: 0.3444
Epoch 2/15
 - 10s - loss: 1.0472 - acc: 0.5845 - val_loss: 1.4331 - val_acc: 0.3579
Epoch 3/15
 - 10s - loss: 1.0010 - acc: 0.6073 - val_loss: 1.3651 - val_acc: 0.3734
Epoch 4/15
 - 9s - loss: 0.9731 - acc: 0.6169 - val_loss: 1.3192 - val_acc: 0.3950
Epoch 5/15
 - 10s - loss: 0.9497 - acc: 0.6314 - val_loss: 1.3159 - val_acc: 0.3847
Epoch 6/15
 - 9s - loss: 0.9382 - acc: 0.6380 - val_

Test Accuracy: 0.5918397407571071
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_1': 0.4, 'lr': 1e-07, 'beta_2': 0.99}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 20s - loss: 2.1018 - acc: 0.1766 - val_loss: 1.8401 - val_acc: 0.3152
Epoch 2/15
 - 9s - loss: 2.0867 - acc: 0.1849 - val_loss: 1.8331 - val_acc: 0.3133
Epoch 3/15
 - 10s - loss: 2.0872 - acc: 0.1811 - val_loss: 1.8264 - val_acc: 0.3110
Epoch 4/15
 - 10s - loss: 2.0727 - acc: 0.1851 - val_loss: 1.8198 - val_acc: 0.3102
Epoch 00004: early stopping
14958/14958 [==============================] - 2s 157us/step
Validation Accuracy: 31.0202%
Validation Loss: 1.8198242240577542
Test Accuracy: 0.2010605391073796
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'b

Epoch 5/15
 - 9s - loss: 1.8936 - acc: 0.2080 - val_loss: 1.6961 - val_acc: 0.2938
Epoch 6/15
 - 10s - loss: 1.8569 - acc: 0.2198 - val_loss: 1.6796 - val_acc: 0.2867
Epoch 7/15
 - 10s - loss: 1.8311 - acc: 0.2253 - val_loss: 1.6684 - val_acc: 0.1570
Epoch 00007: early stopping
14958/14958 [==============================] - 2s 137us/step
Validation Accuracy: 15.7040%
Validation Loss: 1.6683723803493553
Test Accuracy: 0.30041243187509203
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_1': 0.9, 'lr': 0.001, 'beta_2': 0.995}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 22s - loss: 1.2295 - acc: 0.5092 - val_loss: 1.4836 - val_acc: 0.3085
Epoch 2/15
 - 10s - loss: 1.0614 - acc: 0.5809 - val_loss: 1.4830 - val_acc: 0.3430
Epoch 3/15
 - 10s - loss: 1.0085 - acc: 0.6039 - val_loss: 1.3635 - val_acc: 0.3521
Epoch 4/15
 - 9s - loss: 0.9765 - acc: 